In [112]:
#Importing Libraries and Reading Data
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('english')
stop_words = set(stopwords.words('english'))
data = pd.read_csv('data.csv')
data

,Text,Category
0,"In 2018, D'Ernest Johnson wasn't signed by any...",Sports
1,Tom Brady congratulated Aaron Rodgers for beco...,Sports
2,Leading Formula One drivers defended the popul...,Sports
3,Greece's Maria Sakkari will make her debut at ...,Sports
4,"Gymnast Simone Biles, who disclosed her mental...",Sports
5,The NBA is facing another incident involving C...,Sports
6,An Italian DJ claims he suffered a number of i...,Sports
7,Canadian athletes looking to compete for Team ...,Sports
8,It's not often an NBA star can quietly make hi...,Sports
9,With 100 race wins and seven world championshi...,Sports


<h1>Stemming and Removing Stopwords

In [113]:
def stopwords_and_stemming(text):
    text = text.split()
    clean_data = []
    for word in text:
        if word not in stop_words:
            clean_data.append(stemmer.stem(word))
    return ' '.join(clean_data)

data['Text'] = data['Text'].apply(stopwords_and_stemming)
data

,Text,Category
0,"in 2018, d'ernest johnson sign nfl team fail d...",Sports
1,"tom bradi congratul aaron rodger becom ""shareh...",Sports
2,lead formula one driver defend popular netflix...,Sports
3,greec maria sakkari make debut wta final 26-ye...,Sports
4,"gymnast simon biles, disclos mental health str...",Sports
5,the nba face anoth incid involv china ene kant...,Sports
6,an italian dj claim suffer number injuri follo...,Sports
7,canadian athlet look compet team canada beij 2...,Sports
8,it often nba star quiet make way crowd unnotic...,Sports
9,with 100 race win seven world championship nam...,Sports


<h1>Creating Text Classifier Class

In [117]:
class TextClassifier():
    def __init__(self, dataset):
        #Splitting the given data
        txt_train, txt_test, cat_train, cat_test = train_test_split(dataset.Text, dataset.Category, test_size=.2)
        #Creating training set and test set for class
        self.training_set = pd.DataFrame({'Text':txt_train,'Category':cat_train})
        self.test_set = pd.DataFrame({'Text':txt_test,'Category':cat_test})
        self.raw_data = dataset
        print('Model Created...')


    def create_training_table(self):
        for article in self.training_set[self.training_set['Category'] == 'Sports'].Text:
            sports_words = pd.Series(article.split())

        for article in self.training_set[self.training_set['Category'] == 'Business'].Text:
            business_words = pd.Series(article.split())

        for article in self.training_set[self.training_set['Category'] == 'Politics'].Text:
            poli_words = pd.Series(article.split())

        self.training_set = pd.DataFrame({'Sports':sports_words, 'Business':business_words, 'Politics':poli_words})

In [118]:
model = TextClassifier(data)
model.training_set

Model Created...


,Text,Category
24,presid joe biden admit thursday wrong make sta...,Politics
6,an italian dj claim suffer number injuri follo...,Sports
18,suppli chain bottleneck weigh econom growth eu...,Business
20,presid joe biden month circumspect describ beh...,Politics
8,it often nba star quiet make way crowd unnotic...,Sports
26,the hous repres vote thursday hold steve banno...,Politics
11,price consum good rise unit state tangl suppli...,Business
5,the nba face anoth incid involv china ene kant...,Sports
28,the justic depart plan call us secret servic o...,Politics
22,presid joe biden said thursday would support a...,Politics


In [119]:
model.create_training_table()
model.training_set

,Sports,Business,Politics
0,in,the,neera
1,"2018,",question,tanden
2,d'ernest,regul,name
3,johnson,cutting-edg,white
4,sign,drive,hous
...,...,...,...
576,NaN,driver,NaN
577,NaN,like,NaN
578,NaN,distract,NaN
579,NaN,use,NaN
